In [1]:
import torch
import numpy as np
from torch import einsum
from tqdm.auto import tqdm
import seaborn as sns
from transformer_lens import HookedTransformer, ActivationCache, utils, patching
from datasets import load_dataset
from einops import einsum
import pandas as pd
from transformer_lens import utils
from rich.table import Table, Column
from rich import print as rprint
from jaxtyping import Float, Int, Bool
from typing import List, Tuple
from torch import Tensor
import einops
import functools
from transformer_lens.hook_points import HookPoint
# import circuitsvis
from IPython.display import HTML
from plotly.express import line
import plotly.express as px
from tqdm.auto import tqdm
import json
import gc
import plotly.graph_objects as go

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from plotly.subplots import make_subplots
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import load_txt_data, get_mlp_activations, line
import haystack_utils

%reload_ext autoreload
%autoreload 2

In [8]:
model = HookedTransformer.from_pretrained("EleutherAI/pythia-70m",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device)

german_data = haystack_utils.load_json_data("data/german_europarl.json")
english_data = haystack_utils.load_json_data("data/english_europarl.json")


english_activations = {}
german_activations = {}
for layer in range(3, 6):
    english_activations[layer] = get_mlp_activations(english_data[:200], layer, model, mean=False)
    german_activations[layer] = get_mlp_activations(german_data[:200], layer, model, mean=False)


LOG_PROB_THRESHOLD = -7
LAYER_TO_ABLATE = 3
NEURONS_TO_ABLATE = [669]
MEAN_ACTIVATION_ACTIVE = german_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()
MEAN_ACTIVATION_INACTIVE = english_activations[LAYER_TO_ABLATE][:, NEURONS_TO_ABLATE].mean()

def deactivate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_INACTIVE
    return value
deactivate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', deactivate_neurons_hook)]

def activate_neurons_hook(value, hook):
    value[:, :, NEURONS_TO_ABLATE] = MEAN_ACTIVATION_ACTIVE
    return value
activate_neurons_fwd_hooks=[(f'blocks.{LAYER_TO_ABLATE}.mlp.hook_post', activate_neurons_hook)]

all_ignore, not_ignore = haystack_utils.get_weird_tokens(model, plot_norms=False)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
data/german_europarl.json: Loaded 2000 examples with 152 to 2000 characters each.
data/english_europarl.json: Loaded 2000 examples with 165 to 2000 characters each.


  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [15]:
german_losses = []
for prompt in tqdm(german_data):
    original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=None, context_ablation_hooks=deactivate_neurons_fwd_hooks, context_activation_hooks=activate_neurons_fwd_hooks)
    german_losses.append((original_loss, ablated_loss, context_and_activated_loss, only_activated_loss))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [94]:
def get_mlp5_decrease_measure(losses: list[tuple[Float[Tensor, "pos"], Float[Tensor, "pos"], Float[Tensor, "pos"], Float[Tensor, "pos"]]]):
    measure = []
    for original_loss, ablated_loss, context_and_activated_loss, only_activated_loss in losses:
        #pos_wise_diff = ablated_loss - only_activated_loss
        #measure.append(pos_wise_diff.max().item())

        pos_wise_ablated_diff = (ablated_loss - only_activated_loss)
        pos_wise_correct_diff = (only_activated_loss - original_loss)
        pos_wise_correct_diff[pos_wise_correct_diff < 0 ] = 0
        combined = pos_wise_ablated_diff - pos_wise_correct_diff
        combined[original_loss > 7] = 0
        combined[original_loss > ablated_loss] = 0
        measure.append(combined.max().item())
    return measure

measure = get_mlp5_decrease_measure(german_losses)
index = [i for i in range(len(measure))]

sorted_measure = list(zip(index, measure))
sorted_measure.sort(key=lambda x: x[1], reverse=True)

In [95]:
def print_prompt(prompt: str):
    str_token_prompt = model.to_str_tokens(model.to_tokens(prompt))
    original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=None, context_ablation_hooks=deactivate_neurons_fwd_hooks, context_activation_hooks=activate_neurons_fwd_hooks)

    pos_wise_diff = (ablated_loss - only_activated_loss).flatten().cpu().tolist()

    pos_wise_ablated_diff = (ablated_loss - only_activated_loss)
    pos_wise_correct_diff = (only_activated_loss - original_loss)
    pos_wise_diff = (pos_wise_ablated_diff - pos_wise_correct_diff).flatten().cpu().tolist()

    loss_list = [loss.flatten().cpu().tolist() for loss in [original_loss, ablated_loss, context_and_activated_loss, only_activated_loss]]
    loss_names = ["original_loss", "ablated_loss", "context_and_activated_loss", "only_activated_loss"]
    haystack_utils.clean_print_strings_as_html(str_token_prompt[1:], pos_wise_diff, max_value=5, additional_measures=loss_list, additional_measure_names=loss_names)

In [96]:
def average_loss_plot(prompts: list[str], model: HookedTransformer):

    original_losses, ablated_losses, context_and_activated_losses, only_activated_losses = [], [], [], []
    names = ["Original", "Ablated", "Context + MLP5 active", "MLP5 active"]
    for prompt in prompts:
        original_loss, ablated_loss, context_and_activated_loss, only_activated_loss = haystack_utils.get_direct_effect(prompt, model, pos=-1, context_ablation_hooks=deactivate_neurons_fwd_hooks, context_activation_hooks=activate_neurons_fwd_hooks)
        original_losses.append(original_loss)
        ablated_losses.append(ablated_loss)
        context_and_activated_losses.append(context_and_activated_loss)
        only_activated_losses.append(only_activated_loss)

    haystack_utils.plot_barplot([original_losses, ablated_losses, context_and_activated_losses, only_activated_losses], names, ylabel="Loss", title=f"Average loss '{prompts[0][-20:]}'")

## Looking for ngrams

In [97]:
for i, measure in sorted_measure[:10]:
    print_prompt(german_data[i])

In [68]:
# Ansicht
# Verteidig
# Minderheit

In [86]:
print_prompt("Damit entscheide ich mich für die erste Option. Als nächster Punk")

## Ansicht

In [71]:
prompts = [
    "Ich finde ihre Ansicht",
    "Ich teile seine Ansicht",
    "Ich teile deine Ansicht",
    "Die politische Ansicht",
    "Sie teilt die Ansicht",
    "Ich stimme mit deiner Ansicht"
]

for prompt in prompts:
    print_prompt(prompt)

In [72]:
average_loss_plot(prompts, model)

## Voschlägen

In [59]:
# nächster
prompts = ["Ich habe noch einige Fragen zu Ihren Vorschlägen",
    "Ich stimme den Vorschlägen",
    "Kannst du bitte mit deinen Vorschlägen",
    "Laut den Vorschlägen",
    "Die Diskussion wurde mit vielen interessanten Vorschlägen",
    "Sie schrieb einen Brief mit ihren Vorschlägen",
    "Wir werden nach deinen Vorschlägen",
    "Sind Sie mit diesen Vorschlägen",
    "Gemäß den Vorschlägen",
    "Der Kunde war nicht einverstanden mit unseren Vorschlägen",
    "Sie zeigte uns ein Dokument mit ihren Vorschlägen",
    "Das Team arbeitet an den Vorschlägen",
    "Meinen Vorschlägen",
    "Der Ausschuss wird nach den Vorschlägen",
    "Sie war unzufrieden mit den Vorschlägen",
    "Unsere Agentur kam mit neuen Vorschlägen",
    "Haben Sie Änderungen zu den Vorschlägen",
    "Gemäß Ihren Vorschlägen",
    "Ich schrieb einen Bericht mit meinen Vorschlägen",
    "Nach den Vorschlägen",
    "Ich werde gemäß Ihren Vorschlägen",
    "Der Manager war unzufrieden mit den Vorschlägen",
    "Mit diesen Vorschlägen",
    "Der Ausschuss stimmte den Vorschlägen",
    "Der Leiter war sehr zufrieden mit den Vorschlägen",
    "Nach den aktuellen Vorschlägen",
    "Mit ihren innovativen Vorschlägen",
    "Mit einigen Verbesserungen zu Ihren Vorschlägen",
    "Das Team kam mit neuen Vorschlägen",
    "Sie kam mit großartigen Vorschlägen",
    "Ich werde mit meinen Vorschlägen",
    "Sie zeigte sich zufrieden mit den Vorschlägen",
    "Der Direktor war beeindruckt von den Vorschlägen",
    "Die Organisation hat nach Ihren Vorschlägen",
    "Wir haben ein Dokument mit den Vorschlägen",
    "Mit einigen Vorschlägen",
    "Der Lehrer war sehr zufrieden mit den Vorschlägen",
    "Nach Ihren Vorschlägen",
    "In Übereinstimmung mit den Vorschlägen",
    "Sie stimmte den Vorschlägen",
    "Mit diesen neuen Vorschlägen",
    "Ich bin sehr beeindruckt von Ihren Vorschlägen",
    "Das Unternehmen hat nach unseren Vorschlägen",
    "Die Jury war beeindruckt von den Vorschlägen",
    "Die Verwaltung hat nach Ihren Vorschlägen",
    "Das Publikum war sehr zufrieden mit den Vorschlägen",
]


for prompt in prompts[:5]:
    print_prompt(prompt)

In [61]:
average_loss_plot(prompts, model)